In [1]:
! pip install SpeechRecognition pydub langdetect googletrans==4.0.0-rc1 pyaudio

  Using cached httpx-0.13.3-py3-none-any.whl.metadata (25 kB)
  Using cached httpcore-0.9.1-py3-none-any.whl.metadata (4.6 kB)
  Using cached h11-0.9.0-py2.py3-none-any.whl.metadata (8.1 kB)
Using cached httpx-0.13.3-py3-none-any.whl (55 kB)
Using cached httpcore-0.9.1-py3-none-any.whl (42 kB)
Using cached h11-0.9.0-py2.py3-none-any.whl (53 kB)
  Attempting uninstall: h11
    Found existing installation: h11 0.16.0
    Uninstalling h11-0.16.0:
      Successfully uninstalled h11-0.16.0
  Attempting uninstall: httpcore
    Found existing installation: httpcore 1.0.9
    Uninstalling httpcore-1.0.9:
      Successfully uninstalled httpcore-1.0.9
  Attempting uninstall: httpx
    Found existing installation: httpx 0.28.1
    Uninstalling httpx-0.28.1:
      Successfully uninstalled httpx-0.28.1


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
chromadb 1.0.13 requires httpx>=0.27.0, but you have httpx 0.13.3 which is incompatible.
openai 1.90.0 requires httpx<1,>=0.23.0, but you have httpx 0.13.3 which is incompatible.


In [5]:
import speech_recognition as sr
from googletrans import Translator
import time

def speech_to_text():
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        print("\nSpeak now...")
        try:
            recognizer.adjust_for_ambient_noise(source, duration=0.5)
            audio = recognizer.listen(source, timeout=5)
            try:
                # Try English first
                text = recognizer.recognize_google(audio, language="en-IN")
                print("You said (EN):", text)
                return text
            except sr.UnknownValueError:
                # Fall back to Hindi
                text = recognizer.recognize_google(audio, language="hi-IN")
                print("You said (HI):", text)
                return text
        except Exception as e:
            print("Error in listening:", str(e))
            return None

def detect_language(text):
    if not text:
        return "Unknown"
    
    translator = Translator()
    try:
        detected = translator.detect(text)
        if detected.lang == 'hi':
            # Enhanced Hinglish detection
            hinglish_triggers = ["is", "are", "you", "hai", "ho", "the", "my", "main", "hoon", "doing"]
            if any(word in text.lower() for word in hinglish_triggers):
                return "Hinglish"
            return "Hindi"
        elif detected.lang == 'en':
            return "English"
        return "Unknown"
    except Exception as e:
        print("Detection error:", str(e))
        return "Unknown"

def translate_to_english(text):
    if not text:
        return "No text to translate"
    
    translator = Translator()
    try:
        translated = translator.translate(text, dest='en')
        return translated.text
    except Exception as e:
        print("Translation error:", str(e))
        return "Translation failed"

def main():
    max_attempts = 100000  # Prevent infinite loops
    attempt = 0
    
    while attempt < max_attempts:
        attempt += 1
        try:
            text = speech_to_text()
            if not text:
                time.sleep(1)
                continue
                
            lang = detect_language(text)
            print(f"Detected Language: {lang}")
            
            if lang == "Hindi":
                print(f"\nHindi: {text}")
                print(f"English: {translate_to_english(text)}")
            elif lang == "Hinglish":
                print(f"\nHinglish: {text}")
                print(f"English: {translate_to_english(text)}")
            elif lang == "English":
                print(f"\nEnglish: {text}")
                # No translation needed for English
            else:
                print("\nCould not detect language")
            
            print("\n" + "="*40)
            time.sleep(1)  # Add delay between attempts
            
        except KeyboardInterrupt:
            print("\nExiting...")
            break
        except Exception as e:
            print(f"Unexpected error: {str(e)}")
            time.sleep(2) 

if __name__ == "__main__":
    main()


Speak now...
You said (EN): student of CA Foundation
Detected Language: English

English: student of CA Foundation


Speak now...
You said (EN): computer
Detected Language: English

English: computer


Speak now...
You said (EN): aaj din bahut Achcha hai
Detected Language: Hinglish

Hinglish: aaj din bahut Achcha hai
English: it is a nice day today


Speak now...
You said (EN): how an amazing day today thank you
Detected Language: English

English: how an amazing day today thank you


Speak now...

Exiting...
